In [2]:
import pandas as pd
import numpy as np
import math
import json
import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2
#import helpers as h

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
main_cluster = pd.read_csv('main_cluster.csv')
main_cluster = main_cluster.set_index(main_cluster.columns[0])

#rename cluster name to m_1, m_2, etc
main_cluster['cluster'] = main_cluster['cluster'].apply(lambda x: 'm_'+ str(x))

In [5]:
main_cluster.shape

(14808, 93)

In [6]:
main_cluster.head()

,age,income,offer received_bogo_10_10_7,offer received_bogo_10_10_5,offer received_informational_0_0_4,offer received_bogo_5_5_7,offer received_discount_20_5_10,offer received_discount_7_3_7,offer received_discount_10_2_10,offer received_informational_0_0_3,...,gender_F,gender_M,gender_O,member_year_2013,member_year_2014,member_year_2015,member_year_2016,member_year_2017,member_year_2018,cluster
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
0610b486422d4921ae7d2bf64640c50b,55,112000.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,1,0,0,0,0,0,0,1,0,m_7
78afa995795e4d85b5d9ceeca43f5fef,75,100000.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,1,0,0,0,0,0,0,1,0,m_5
e2127556f4f64592b11af22de27a7932,68,70000.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,...,0,1,0,0,0,0,0,0,1,m_5
389bc3fa690240e798340f5a15918d5c,65,53000.0,0.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,...,0,1,0,0,0,0,0,0,1,m_5
2eeac8d8feae4a8cad5a6af0499a211d,58,51000.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,...,0,1,0,0,0,0,0,1,0,m_1


In [7]:
# check for null data
main_cluster.isnull().sum().sum()

0

In [8]:

features = ['age',
 'avg_spending',
 'gender_F',
 'gender_M',
 'gender_O',
 'income',
 'informational_received',
 'invalid',
 'member_days_since',
 'member_year_2013',
 'member_year_2014',
 'member_year_2015',
 'member_year_2016',
 'member_year_2017',
 'member_year_2018',
 'offer received_bogo_10_10_5',
 'offer received_bogo_10_10_7',
 'offer received_bogo_5_5_5',
 'offer received_bogo_5_5_7',
 'offer received_discount_10_2_10',
 'offer received_discount_10_2_7',
 'offer received_discount_20_5_10',
 'offer received_discount_7_3_7',
 'offer received_informational_0_0_3',
 'offer received_informational_0_0_4',
 'offer viewed_bogo_10_10_5',
 'offer viewed_bogo_10_10_7',
 'offer viewed_bogo_5_5_5',
 'offer viewed_bogo_5_5_7',
 'offer viewed_discount_10_2_10',
 'offer viewed_discount_10_2_7',
 'offer viewed_discount_20_5_10',
 'offer viewed_discount_7_3_7',
 'rate_offer_viewed_bogo_10_10_5',
 'rate_offer_viewed_bogo_10_10_7',
 'rate_offer_viewed_bogo_5_5_5',
 'rate_offer_viewed_bogo_5_5_7',
 'rate_offer_viewed_discount_10_2_10',
 'rate_offer_viewed_discount_10_2_7',
 'rate_offer_viewed_discount_20_5_10',
 'rate_offer_viewed_discount_7_3_7',
 'rate_viewed_bogo',
 'rate_viewed_discount',
 'sum_spending',
 'transaction_count', 'cluster']

X = main_cluster[features]

target_cols = ['rate_completed_received_bogo',
 'rate_completed_received_discount','rate_viewed_informational']

y = main_cluster[target_cols]

In [9]:
X.shape

(14808, 46)

In [10]:
X_one_hot = pd.get_dummies(X)

def one_hot(df):
    return pd.get_dummies(df)

In [11]:
X_one_hot.columns

Index(['age', 'avg_spending', 'gender_F', 'gender_M', 'gender_O', 'income',
       'informational_received', 'invalid', 'member_days_since',
       'member_year_2013', 'member_year_2014', 'member_year_2015',
       'member_year_2016', 'member_year_2017', 'member_year_2018',
       'offer received_bogo_10_10_5', 'offer received_bogo_10_10_7',
       'offer received_bogo_5_5_5', 'offer received_bogo_5_5_7',
       'offer received_discount_10_2_10', 'offer received_discount_10_2_7',
       'offer received_discount_20_5_10', 'offer received_discount_7_3_7',
       'offer received_informational_0_0_3',
       'offer received_informational_0_0_4', 'offer viewed_bogo_10_10_5',
       'offer viewed_bogo_10_10_7', 'offer viewed_bogo_5_5_5',
       'offer viewed_bogo_5_5_7', 'offer viewed_discount_10_2_10',
       'offer viewed_discount_10_2_7', 'offer viewed_discount_20_5_10',
       'offer viewed_discount_7_3_7', 'rate_offer_viewed_bogo_10_10_5',
       'rate_offer_viewed_bogo_10_10_7', 'rate_

In [12]:
from sklearn.preprocessing import StandardScaler, RobustScaler

def scaling(features):
    scale = StandardScaler()
    scale.fit(features)
    features_std = pd.DataFrame(scale.transform(features), index= features.index, columns=features.columns)
    
    return features_std

X_std = scaling(X_one_hot)

In [13]:
def get_train_test(features, target):
    
    """ Split : Train - Test """

    from sklearn.model_selection import train_test_split as tts

    X_train, X_test, y_train, y_test = tts(features, target, test_size=0.33, random_state=42)
    
    return X_train, X_test, y_train, y_test

In [14]:
# as feature scaling will be included in model pipeline
# the data will be split after performing one hot encoding.

X_train, X_test, y_train, y_test = get_train_test(X_one_hot, y)

In [15]:
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.metrics import  mean_squared_error
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler

In [16]:
from sklearn.linear_model import LinearRegression


seed=42
np.random.seed(seed)
estimators = [('standardize', StandardScaler()),
             ('reg', MultiOutputRegressor(LinearRegression()))]

pipeline = Pipeline(estimators)
kfold = KFold(n_splits=10, random_state=seed)
results = cross_validate(pipeline, X_train, y_train, cv=kfold, scoring=['r2','neg_mean_squared_error'])

r2_arr = results['test_r2']
mse_arr = results['test_neg_mean_squared_error']*-1

print(f"R^2: {r2_arr.mean():.4f} with stdev {r2_arr.std():.4f}")
print(f"MSE: {mse_arr.mean():.4f} with stdev {mse_arr.std():.4f}")

R^2: 0.6100 with stdev 0.0120
MSE: 0.0783 with stdev 0.0020


In [ ]:
# get model list
models = h.get_models_multioutput()
# evaluate models
results = h.evaluate_models(X_train, y_train, models, metric='neg_mean_squared_error')